In [3]:
from flask import Flask, render_template, request
from flask_uploads import UploadSet, IMAGES, configure_uploads

app = Flask(__name__)

photos = UploadSet('photos', [IMAGES, 'pgm'])
app.config['UPLOADED_PHOTOS_DEST'] = 'static/img'
configure_uploads(app, photos)

import numpy as np
import os
from sklearn.externals import joblib
from skimage import io
from skimage.color import rgb2gray
from skimage import exposure, feature, transform
from flask import jsonify, json

svc = joblib.load('./clf/clf_svc_hog.pkl')

def feature_extract(image):
    gray_img = rgb2gray(image)
    (image_feat, hogImage) = feature.hog(gray_img, orientations=9, pixels_per_cell=(8,8),
    cells_per_block=(2,2), transform_sqrt=True, visualise=True)
    image_feat = image_feat.reshape(1, -1)
    return image_feat

def predict_face(im_path):
    image =io.imread(im_path)
    image_feat = feature_extract(image)
    pred_label = svc.predict(image_feat)[0]
    result = json.dumps({'results':pred_label},indent=4)
    return result

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/upload', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST' and 'photo' in request.files:
        filename = photos.save(request.files['photo'])
        f = os.path.join(app.config['UPLOADED_PHOTOS_DEST'],filename)
        return predict_face(f)
#         return predict_face('static/img'+'/'+filename)
    return render_template('upload.html')


/home/shravan/anaconda3/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator LinearSVC from version 0.19.0 when using version 0.18.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [ ]:
app.run(port=5000)

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Sep/2017 22:40:12] "GET /upload HTTP/1.1" 200 -
/home/shravan/anaconda3/lib/python3.6/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)
127.0.0.1 - - [25/Sep/2017 22:40:16] "POST /upload HTTP/1.1" 200 -
